# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import set_config
set_config(
   assume_finite=True,  # disable validation
)

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)

In [3]:
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(df[~df['related'].isin([0,1])].index)

In [5]:
df = df.dropna()

In [6]:
df = df.drop_duplicates()

In [7]:
df.shape

(10038, 41)

In [8]:
related_id = list(df.columns).index("related")

In [9]:
X = df.message
Y = df[df.columns[related_id:]]

In [10]:
Y["related"].unique()

array([1, 0])

In [11]:
Y[~Y['related'].isin([0,1])]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\Max\AppData\Local\Temp\ipykernel_53628\4253155143.py:2: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [14]:
pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021550D12DE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Max\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Max\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(x_train, y_train)

c:\Users\Max\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000021550D12DE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline.predict(x_test)

In [18]:
cols = list(Y.columns)

In [20]:
y_test["related"].unique()

array([1, 0])

In [21]:
set(y_pred[:,index])

{np.int64(0), np.int64(1)}

In [22]:
for col in cols:
    index = cols.index(col)
    print(col, ':')
    print(classification_report(y_test[col], y_pred[:,index], target_names=Y.columns))
    print('----------------------------------------------------------------------')

related :


ValueError: Number of classes, 2, does not match size of target_names, 36. Try specifying the labels parameter

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000023222A82D40>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000023222A82D40>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [ ]:
parameters = {
    'clf__estimator__n_estimators': [20, 50, 100],
    'clf__estimator__min_samples_split': [2],
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

cv.fit(x_train, y_train)
best_params = cv.best_params_
best_model = cv.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = best_model.predict(x_test)

In [ ]:
for col in cols:
    index = cols.index(col)
    print(col, ':')
    print(classification_report(y_test[col], y_pred[:,index], target_names=cols))
    print('----------------------------------------------------------------------')

related :
                        precision    recall  f1-score   support

               related       0.74      0.28      0.41      1587
               request       0.80      0.97      0.88      4912
                 offer       0.53      0.15      0.23        55

           avg / total       0.78      0.79      0.76      6554

----------------------------------------------------------------------
request :
                        precision    recall  f1-score   support

               related       0.89      0.99      0.94      5434
               request       0.90      0.43      0.58      1120

           avg / total       0.89      0.89      0.88      6554

----------------------------------------------------------------------
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6525
               request       0.00      0.00      0.00        29

           avg / total       0.99      1.00      0.99 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                        precision    recall  f1-score   support

               related       0.92      1.00      0.96      6043
               request       0.56      0.03      0.06       511

           avg / total       0.90      0.92      0.89      6554

----------------------------------------------------------------------
medical_products :
                        precision    recall  f1-score   support

               related       0.95      1.00      0.98      6227
               request       0.80      0.04      0.07       327

           avg / total       0.94      0.95      0.93      6554

----------------------------------------------------------------------
search_and_rescue :
                        precision    recall  f1-score   support

               related       0.97      1.00      0.99      6374
               request       0.78      0.04      0.07       180

           avg / total       0.97      0.97      0.96      6554

------------------------------------------

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


                        precision    recall  f1-score   support

               related       0.95      1.00      0.98      6150
               request       0.89      0.27      0.42       404

           avg / total       0.95      0.95      0.94      6554

----------------------------------------------------------------------
food :
                        precision    recall  f1-score   support

               related       0.93      0.99      0.96      5823
               request       0.89      0.40      0.55       731

           avg / total       0.92      0.93      0.91      6554

----------------------------------------------------------------------
shelter :
                        precision    recall  f1-score   support

               related       0.93      1.00      0.96      5979
               request       0.88      0.25      0.39       575

           avg / total       0.93      0.93      0.91      6554

----------------------------------------------------------------

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

# Assuming you have a trained model called 'model'
# and you want to export it as a pickle file

# Save the model as a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.